In [0]:
import pandas as pd

v1 = spark.table("workspace.default.athletes").toPandas()
display(v1.head())

athlete_id,name,region,team,affiliate,gender,age,height,weight,fran,helen,grace,filthy50,fgonebad,run400,run5k,candj,snatch,deadlift,backsq,pullups,eat,train,background,experience,schedule,howlong
2554.0,Pj Ablang,South West,Double Edge,Double Edge CrossFit,Male,24.0,70.0,166.0,null,null,null,null,null,null,null,220.0,null,400.0,305.0,null,null,I workout mostly at a CrossFit Affiliate|I have a coach who determines my programming|I record my workouts|,I played youth or high school level sports|I regularly play recreational sports|,I began CrossFit with a coach (e.g. at an affiliate)|I have attended one or more specialty courses|I have had a life changing experience due to CrossFit|,I do multiple workouts in a day 2x a week|,4+ years|
3517.0,Derek Abdella,null,null,null,Male,42.0,70.0,190.0,null,null,null,null,0.0,null,null,null,null,null,null,null,null,I have a coach who determines my programming|I record my workouts|,I played youth or high school level sports|,I began CrossFit with a coach (e.g. at an affiliate)|I have attended one or more specialty courses|I have had a life changing experience due to CrossFit|,I do multiple workouts in a day 2x a week|,4+ years|
4691.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
5164.0,Abo Brandon,Southern California,LAX CrossFit,LAX CrossFit,Male,40.0,67.0,null,211.0,645.0,300.0,null,196.0,null,null,245.0,200.0,375.0,325.0,25.0,I eat 1-3 full cheat meals per week|,I workout mostly at a CrossFit Affiliate|I have a coach who determines my programming|I record my workouts|,I played youth or high school level sports|,I began CrossFit by trying it alone (without a coach)|I began CrossFit with a coach (e.g. at an affiliate)|I have completed the CrossFit Level 1 certificate course|I have attended one or more specialty courses|I have had a life changing experience due to CrossFit|I train other people|,I usually only do 1 workout a day|,4+ years|
5286.0,Bryce Abbey,null,null,null,Male,32.0,65.0,149.0,206.0,465.0,null,1053.0,null,null,1081.0,205.0,150.0,null,325.0,50.0,I eat quality foods but don't measure the amount|,I workout mostly at a CrossFit Affiliate|I incorporate CrossFit.com workouts|I write my own programming|I record my workouts|,I played college sports|,I began CrossFit by trying it alone (without a coach)|I have completed the CrossFit Level 1 certificate course|I train other people|,I usually only do 1 workout a day|I strictly schedule my rest days|,1-2 years|


In [0]:
import numpy as np
import pandas as pd

def clean_data(data):
    # Remove not relevant columns
    data = data.dropna(subset=['region','age','weight','height','howlong','gender','eat', \
                               'train','background','experience','schedule','howlong', \
                               'deadlift','candj','snatch','backsq','experience',\
                               'background','schedule','howlong'])
    data = data.drop(columns=['affiliate','team','name','fran','helen','grace',\
                              'filthy50','fgonebad','run400','run5k','pullups','train'])

    # Remove Outliers

    data = data[data['weight'] < 1500]
    data = data[data['gender'] != '--']
    data = data[data['age'] >= 18]
    data = data[(data['height'] < 96) & (data['height'] > 48)]

    data = data[(data['deadlift'] > 0) & (data['deadlift'] <= 1105)|((data['gender'] == 'Female') \
                 & (data['deadlift'] <= 636))]
    data = data[(data['candj'] > 0) & (data['candj'] <= 395)]
    data = data[(data['snatch'] > 0) & (data['snatch'] <= 496)]
    data = data[(data['backsq'] > 0) & (data['backsq'] <= 1069)]

    # Clean Survey Data

    decline_dict = {'Decline to answer|': np.nan}
    data = data.replace(decline_dict)
    data = data.dropna(subset=['background','experience','schedule','howlong','eat'])

    return data

In [0]:
v2 = v1
v2 = clean_data(v1)

display(v2.head())

athlete_id,region,gender,age,height,weight,candj,snatch,deadlift,backsq,eat,background,experience,schedule,howlong
21269.0,Southern California,Male,30.0,71.0,200.0,235.0,175.0,385.0,315.0,I eat whatever is convenient|,I played youth or high school level sports|I played college sports|I regularly play recreational sports|,I began CrossFit by trying it alone (without a coach)|I have had a life changing experience due to CrossFit|,I do multiple workouts in a day 1x a week|I typically rest 4 or more days per month|,1-2 years|
21685.0,Africa,Male,28.0,70.0,176.0,187.0,134.0,335.0,254.0,I eat 1-3 full cheat meals per week|,I have no athletic background besides CrossFit|,I began CrossFit with a coach (e.g. at an affiliate)|I have had a life changing experience due to CrossFit|,I do multiple workouts in a day 1x a week|,2-4 years|
25464.0,North East,Male,35.0,68.0,225.0,285.0,205.0,440.0,405.0,I eat quality foods but don't measure the amount|,I played youth or high school level sports|,I began CrossFit with a coach (e.g. at an affiliate)|,I typically rest 4 or more days per month|,2-4 years|
43767.0,North Central,Male,36.0,71.0,199.0,267.0,212.0,485.0,390.0,I eat quality foods but don't measure the amount|,I played youth or high school level sports|I played college sports|,I began CrossFit with a coach (e.g. at an affiliate)|I have completed the CrossFit Level 1 certificate course|I have had a life changing experience due to CrossFit|I train other people|,I do multiple workouts in a day 3+ times a week|I strictly schedule my rest days|I typically rest 4 or more days per month|,1-2 years|
55504.0,North East,Male,36.0,64.0,155.0,245.0,180.0,415.0,385.0,I eat strict Paleo|,I played youth or high school level sports|I played college sports|I regularly play recreational sports|,I began CrossFit by trying it alone (without a coach)|I have completed the CrossFit Level 1 certificate course|I have attended one or more specialty courses|I have had a life changing experience due to CrossFit|I train other people|,I do multiple workouts in a day 2x a week|I strictly schedule my rest days|I typically rest 4 or more days per month|,4+ years|


Select the 2 versions of features

In [0]:
from pyspark.sql.functions import col, when

feature_df_v1 = v2[["athlete_id", "gender", "age", "height", "weight", "eat", "background", "schedule", "howlong"]]

v2["BMI"] = v2["weight"] / ((v2["height"] / 100) ** 2)
feature_df_v2 = v2[[
    "athlete_id", "gender", "age", "BMI"
]].copy()

Set up the Feature Store

In [0]:
%pip install databricks-feature-store --upgrade

INFO: pip is looking at multiple versions of google-api-core to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of googleapis-common-protos to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 752.6/752.6 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 108.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 641.0/641.0 kB 29.4 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.4
    Not uninstalling protobuf at /databricks/python3/lib/python3.12/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-6aa9e73d-89b5-46ab-916e-97738d5cd6c4
    Can't uninstall 'protobuf'. No files were found to uninstall.
  Attempting uninstall: googleapis-common-protos
    Found existing installation: googleapis-common-protos 1.65.0
  

In [0]:
from databricks import feature_store

fs = feature_store.FeatureStoreClient()

fs.create_table(
    name="workspace.default.athlete_features_v1",
    primary_keys=["athlete_id"],
    df=feature_df_v1,
    description="Baseline features for athletes (v2) with demographics and habits."
)

fs.create_table(
    name="workspace.default.athlete_features_v2",
    primary_keys=["athlete_id"],
    df=feature_df_v2,
    description="Simplified athlete features v2 with BMI"
)

---------------------------------------------------------------------------
ImportError                               Traceback (most recent call last)
File <command-4984514826154115>, line 1
----> 1 from databricks import feature_store
      3 fs = feature_store.FeatureStoreClient()
      5 fs.create_table(
      6     name="workspace.default.athlete_features_v1",
      7     primary_keys=["athlete_id"],
      8     df=feature_df_v1,
      9     description="Baseline features for athletes (v2) with demographics and habits."
     10 )

ImportError: cannot import name 'feature_store' from 'databricks' (/databricks/python_shell/lib/databricks/__init__.py)